<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [4]</a>'.</span>

In [1]:
# Parameters
data_file = "../../data/experiments/pilot_random1_player_round_slim.csv"
device = "cpu"
experiment_names = ["trail_rounds_2"]
fraction_training = 1.0
labels = {"model": "rnn_edge"}
mask_name = "manager_valid"
model_args = {
    "add_edge_model": True,
    "add_global_model": False,
    "add_rnn": True,
    "hidden_size": 5,
    "u_encoding": [
        {"encoding": "numeric", "n_levels": 16, "name": "round_number"},
        {"etype": "float", "name": "prev_common_good", "norm": 32},
    ],
    "x_encoding": [
        {"encoding": "numeric", "n_levels": 21, "name": "contributions"},
        {"encoding": "numeric", "n_levels": 31, "name": "prev_punishments"},
        {"etype": "bool", "name": "valid"},
        {"etype": "bool", "name": "prev_manager_valid"},
    ],
    "y_levels": 31,
    "y_name": "punishments",
}
model_name = "graph"
n_cross_val = 6
n_player = 4
optimizer_args = {"lr": 0.0005, "weight_decay": 1e-05}
output_path = "/mnt/beegfs/home/brinkmann/repros/algorithmic-institutions/data/training/am_1/train/model_rnn_edge"
shuffle_features = [
    "prev_punishments",
    "prev_contributions",
    "prev_common_good",
    "prev_valid",
]
train_args = {"batch_size": 10, "clamp_grad": 1, "epochs": 5000, "eval_period": 10}


In [2]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import torch as th
from aimanager.generic.data import create_syn_data, create_torch_data, get_cross_validations
from aimanager.artificial_humans import AH_MODELS
from aimanager.artificial_humans.evaluation import Evaluator
from aimanager.utils.array_to_df import using_multiindex
from aimanager.generic.graph_encode import create_fully_connected
from torch_geometric.data import Batch
from torch_geometric.loader import DataLoader

output_path = os.path.join(output_path, 'data')

/mnt/beegfs/home/brinkmann/repros/algorithmic-institutions/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv(data_file)

df = df[df['experiment_name'].isin(experiment_names)]

data, default_values = create_torch_data(df)
# syn_data = create_syn_data(n_contribution=21, n_punishment=31, default_values=default_values)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [4]:
th_device = th.device(device)

metrics = []
confusion_matrix = []
syn_pred = []
ev = Evaluator()

th_device = th.device(device)

syn_index = ['prev_punishments', 'prev_contributions']
edge_index = create_fully_connected(n_player)


def shuffle_feature(data, feature_name):
    data = {**data}
    data[feature_name] = data[feature_name][th.randperm(len(data[feature_name]))]
    return data

for i, (train_data, test_data) in enumerate(get_cross_validations(data, n_cross_val, fraction_training)):
    model = AH_MODELS[model_name](default_values=default_values, **model_args).to(th_device)

    train_data_ = model.encode(train_data, mask=mask_name, edge_index=edge_index)
    if test_data is not None:
        test_data_ = model.encode(test_data, mask=mask_name, edge_index=edge_index)
    # syn_data_ = model.encode(syn_data, mask=None, y_encode=False, info_columns=syn_index, edge_index=edge_index)

    # syn_df = using_multiindex(
    #     Batch.from_data_list(syn_data_)['info'].detach().cpu().numpy(), ['idx', 'round_number'], syn_index)

    optimizer = th.optim.Adam(model.parameters(), **optimizer_args)
    loss_fn = th.nn.CrossEntropyLoss(reduction='none')
    sum_loss = 0
    n_steps = 0

    for e in range(train_args['epochs']):
        ev.set_labels(cv_split=i, epoch=e)
        model.train()
        for j, batch_data in enumerate(iter(DataLoader(train_data_, shuffle=True, batch_size=train_args['batch_size']))):
            optimizer.zero_grad()
            py = model(batch_data).flatten(end_dim=-2)
            y_true = batch_data['y_enc'].flatten(end_dim=-2)
            mask = batch_data['mask'].flatten()
            loss = loss_fn(py, y_true)
            loss = (loss * mask).sum() / mask.sum()

            loss.backward()

            if train_args['clamp_grad']:
                for param in model.parameters():
                    param.grad.data.clamp_(-train_args['clamp_grad'], train_args['clamp_grad'])
            optimizer.step()
            sum_loss += loss.item()
            n_steps +=1
        
        last_epoch = e == (train_args['epochs'] - 1)

        if (e % train_args['eval_period'] == 0) or last_epoch:
            avg_loss = sum_loss/n_steps
            print(f'CV {i} | Epoch {e} | Loss {avg_loss}')
            ev.add_loss(avg_loss)

            ev.eval_set(model, train_data_, calc_confusion=last_epoch, set='train')
            if test_data is not None:
                ev.eval_set(model, test_data_, calc_confusion=last_epoch, set='test')
                for sf in shuffle_features:
                    shuffled_data = shuffle_feature(test_data, sf)
                    shuffled_data = model.encode(shuffled_data, mask=mask_name, edge_index=edge_index)
                    ev.eval_set(model, shuffled_data, calc_confusion=False, set='test', shuffle_feature=sf)
            sum_loss = 0
            n_steps = 0
    # ev.eval_syn(model, syn_data_, syn_df)

ev.save(output_path, labels)
model_path = os.path.join(output_path, 'model.pt')
model.save(model_path)

CV 0 | Epoch 0 | Loss 3.254797101020813


CV 0 | Epoch 10 | Loss 3.1031448632478713


CV 0 | Epoch 20 | Loss 2.725128224492073


CV 0 | Epoch 30 | Loss 2.3809281796216966


CV 0 | Epoch 40 | Loss 2.1083481878042223


...

CV 0 | Epoch 90 | Loss 1.7579564578831195


CV 0 | Epoch 100 | Loss 1.7634853765368461


CV 0 | Epoch 110 | Loss 1.701007816195488


CV 0 | Epoch 120 | Loss nan


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0